# Детектор эмоций

В этом модуле производиться детекция лиц с веб камеры и распознование эмоции на лицах

In [1]:
import cv2

import numpy as np

import emo_utils as utils
import emo_classificator_model as ecm

### Загрузка классификатора эмоций

In [2]:
#Получаем пути к директориям и файлам проекта
paths = utils.get_paths()

In [3]:
#Путь к сохраненной модели
model_path = paths['model_dir'] + '/emo_classificator_best_210105121553_resnet50_e20_aug1_acc44.h5'


In [4]:
#загрузим названия эмоций (классов)
emo_classes_names = []
with open(paths['class_names_path'], 'r') as f:
    emo_classes_names = f.read().split(',')
    
emo_classes_names

['anger',
 'contempt',
 'disgust',
 'fear',
 'happy',
 'neutral',
 'sad',
 'surprise',
 'uncertain']

In [5]:
#Создадим классификатор эмоций

base_model_names = ['resnet50', 'vgg16', 'xception']

#тип базовой модели
base_model = None
for model_name in base_model_names:
    if model_path.find(model_name) > -1:
        base_model = model_name
        break
    #if

emo_classifier = ecm.EmoClassifier (model_path, base_model, emo_classes_names)

### Распознование эмоций по лицам с веб-камеры

Для завершения нажмите **Escape**

In [7]:
#Детекция лиц на веб камере и определение эмоций

#Цвет рамки лица (RGB)
frame_color = (0, 255, 0)
#Цвет подписи эмоции (RGB)
label_color = (0, 255, 0)

#Шрифт подписи эмоции
font = cv2.FONT_HERSHEY_TRIPLEX
# размер подписи эмоции
font_scale = 0.7
#Частота (количестыо кадров) определения эмоции 
freq_emo_class = 5

# загружаем настройки opencv для детекции лица
face_cascade = cv2.CascadeClassifier(paths['model_dir'] + '/haarcascade_frontalface_alt.xml')

try:
    # захватываем изображение с веб камеры
    cap = cv2.VideoCapture(0)

    cnt = 0
    amo_label = ''
    while True:
        # получаем картинку с камеры
        _, img = cap.read()
        # конвертируем изображение в черно-белые (серые) цвета
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Детектируем лица
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        
        # Отрисовка рамок вокруг каждого лица
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x+w, y+h), frame_color, 2)
        
            #вырезаем лицо по рамке
            crop_img = img[y:y+h, x:x+w]
        
            if cnt == freq_emo_class:
                
                #Подготавливаем изображение для отправки в модель
                preprocessed_img = cv2.resize(crop_img, 
                                              dsize=(emo_classifier.image_size, emo_classifier.image_size), 
                                              interpolation = cv2.INTER_CUBIC)
                
                preprocessed_img = np.asarray(preprocessed_img)
                preprocessed_img = np.expand_dims(preprocessed_img, axis=0)
        
                #получим предсказания
                amo_label = emo_classifier.predict (emo_classifier.preprocess_input_func(preprocessed_img))
                
            #if
          
            cv2.putText(img, amo_label, (x - 3, y - 2), font, font_scale, label_color)
    
        #for
        
        if cnt == freq_emo_class:
            cnt = 0
        else:
            cnt = cnt + 1
        
        # вывод изображения
        cv2.imshow('img', img)
        # Заканчиваем, если была нажата кнопка "Escape"
        k = cv2.waitKey(30) & 0xff
        if k==27:
            break
        
    #while
#try
finally:
    cap.release()
    cv2.destroyAllWindows()
#finally